In [1]:
import tensorflow as tf
import tensorflow.keras.models as KM
import tensorflow.keras.layers as KL
import tensorflow.keras.backend as K
import os
import numpy as np
import matplotlib.pyplot as plt
import time

import glob
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Conv2DTranspose, Dropout, ReLU, Input, Concatenate, ZeroPadding2D
from tensorflow.keras.optimizers import Adam

In [2]:
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer,InputLayer, Input,Reshape, Conv2D, Conv2DTranspose,Embedding,Bidirectional,\
Dense, Flatten,BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D,MaxPooling2D,Dropout,Concatenate,\
Lambda


In [3]:
from tensorflow.keras.optimizers import Adam,RMSprop,Adadelta,SGD
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.initializers import RandomNormal

In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [5]:
import os
import sys


In [2]:
import tensorflow as tf
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))

In [3]:
devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(devices[0] ,enable=True)

In [8]:
BATCH_SIZE = 1
IMAGE_SIZE = 512

In [9]:
label = ['2.5','3.0','3.5','4.0']
data = [[0,1,2,3]]

def indices_to_one_hot(data, nb_classes):
    """Converting exposure labels to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]



In [10]:
one_hot = indices_to_one_hot(data,4)

In [11]:
one_hot

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [12]:
#This function reads the concatenated image and take input and real image separately
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    w = tf.shape(image)[1]
    w = w//2
    real_image = image[:, :w, :]
    input_image = image[:, w:, :]
    
    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)
    return input_image, real_image

In [13]:
#Normalizing the images
def normalize(input_image, real_image):
    input_image = (input_image / 255)
    real_image = (real_image / 255)
    return input_image, real_image

In [14]:
#Resizing the images to 512x512
def resize(input_image, real_image):
    input_image = tf.image.resize(input_image, [IMAGE_SIZE, IMAGE_SIZE], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [IMAGE_SIZE, IMAGE_SIZE], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image, real_image

In [15]:
def load_train_images(image_path):
    input_image, real_image = load(image_path)
    input_image, real_image = resize(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)
    return input_image,real_image

In [16]:
def load_test_image(image_path):
    input_image, real_image = load(image_path)
    input_image, real_image = resize(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)
    return input_image, real_image

In [17]:
#These folders consists of different folder for different exposures 
#Images in these folder are images created by concatenating Low light image with its corresponding normal light image

path = "rellisur_train_"
path_test = "rellisur_test_"

In [4]:
#Here we create corresponding labels for the images for training
for i in range(len(one_hot)):
    globals()[f"train_dataset_{i}"] = tf.data.Dataset.list_files(path + "/train_"+ label[i] + "/*.png")
    globals()[f"train_dataset_{i}"]= globals()[f"train_dataset_{i}"].map(load_train_images)
    globals()[f"label_{i}"]= np.full((len(globals()[f"train_dataset_{i}"]),4), one_hot[i])
    globals()[f"label_{i}"]= tf.cast(globals()[f"label_{i}"], tf.int64)
    globals()[f"label_{i}"]= tf.data.Dataset.from_tensor_slices(globals()[f"label_{i}"])

In [19]:
train_dataset = train_dataset_0.concatenate(train_dataset_1)
train_dataset = train_dataset.concatenate(train_dataset_2)
train_dataset = train_dataset.concatenate(train_dataset_3)

In [20]:
train_label = label_0.concatenate(label_1)
train_label = train_label.concatenate(label_2)
train_label = train_label.concatenate(label_3)

In [21]:
zip_train_dataset = tf.data.Dataset.zip((train_dataset, train_label))

In [24]:
for i in range(len(one_hot)):
    globals()[f"test_dataset_{i}"] = tf.data.Dataset.list_files(path_test + "/test_"+ label[i] + "/*.png")
    globals()[f"test_dataset_{i}"]= globals()[f"test_dataset_{i}"].map(load_test_image)
    globals()[f"test_label_{i}"]= np.full((len(globals()[f"test_dataset_{i}"]),4), one_hot[i])
    globals()[f"test_label_{i}"]= tf.cast(globals()[f"test_label_{i}"], tf.int64)
    globals()[f"test_label_{i}"]= tf.data.Dataset.from_tensor_slices(globals()[f"test_label_{i}"])

In [25]:
test_dataset = test_dataset_0.concatenate(test_dataset_1)
test_dataset = test_dataset.concatenate(test_dataset_2)
test_dataset = test_dataset.concatenate(test_dataset_3)

In [26]:
test_label = test_label_0.concatenate(test_label_1)
test_label = test_label.concatenate(test_label_2)
test_label = test_label.concatenate(test_label_3)

In [27]:
zip_test_dataset_0 = tf.data.Dataset.zip((test_dataset_0, test_label_0))
zip_test_dataset_1 = tf.data.Dataset.zip((test_dataset_1, test_label_1))
zip_test_dataset_2 = tf.data.Dataset.zip((test_dataset_2, test_label_2))
zip_test_dataset_3 = tf.data.Dataset.zip((test_dataset_3, test_label_3))

In [28]:
# Here we are zipping the all the data to create a single dataset
zip_test_dataset = tf.data.Dataset.zip((test_dataset, test_label))

In [30]:
def depth_wise_concatenate(input_img, input_label):
            # one hot vector is spatially replicated and concatenated with the input image
            label = Lambda(lambda x: K.repeat(x, 512**2))(input_label)
            label = Reshape((512, 512, 4))(label)
            x = Concatenate()([input_img, label])  
            return x 

In [31]:
# This is a part of the generator used to downsample the image
def downsample(filters, size, batchnorm = True):
    init = tf.random_normal_initializer(0.,0.02)
    result = Sequential()
    result.add(Conv2D(filters, size, strides = 2, padding = "same", kernel_initializer = init, use_bias = False))
    if batchnorm == True:
        result.add(BatchNormalization())
        
    result.add(LeakyReLU())
    return result

In [32]:
def upsample(filters, size, dropout = False):
    init = tf.random_normal_initializer(0, 0.02)
    result = Sequential()
    result.add(Conv2DTranspose(filters, size, strides = 2, padding = "same", kernel_initializer = init, use_bias = False))
    result.add(BatchNormalization())
    if dropout == True:
        result.add(Dropout(0.5))
    result.add(ReLU())
    return result

In [32]:
def generator():
    inputs = Input(shape = [IMAGE_SIZE, IMAGE_SIZE, 3])
    label = Input(shape=(4,))
    input = depth_wise_concatenate(inputs, label) 
    
    down_stack = [
        downsample(64, 4, batchnorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4)
    ]
    
    
    up_stack = [
        upsample(512, 4, dropout=True),
        upsample(512, 4, dropout=True),
        upsample(512, 4),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]
    init = tf.random_normal_initializer(0., 0.02)
    last = Conv2DTranspose(3, 4, strides = 2, padding = "same", kernel_initializer = init, activation ="tanh")
    x = input
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])
    
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = Concatenate()([x, skip])
    
    x = last(x)
    return Model(inputs = [inputs,label], outputs = x)

In [33]:
gen = generator()
gen.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 lambda (Lambda)                (None, 262144, 4)    0           ['input_2[0][0]']                
                                                                                                  
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 reshape (Reshape)              (None, 512, 512, 4)  0           ['lambda[0][0]']             

In [33]:
from tensorflow.keras.losses import BinaryCrossentropy
loss_function = BinaryCrossentropy(from_logits=True)

In [35]:
# Defining vgg perceptual loss
selected_layers = ["block1_conv2","block2_conv2","block3_conv4","block4_conv4","block5_conv4"]
selected_layer_weights = [0.1,0.1,1.0,1.0,1.0]

vgg = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(512,512,3))
vgg.trainable = False
outputs = [vgg.get_layer(l).output for l in selected_layers]
model = Model(vgg.input, outputs)
rc_loss = None

@tf.function
def perceptual_loss(target , gen_output):
    h1_list = model(target)
    h2_list = model(gen_output)
    global rc_loss
    if rc_loss is None:
        rc_loss = 0.0
    
    for h1, h2, weight in zip(h1_list, h2_list, selected_layer_weights):
        h1 = tf.keras.backend.batch_flatten(h1)
        h2 = tf.keras.backend.batch_flatten(h2)
        rc_loss = rc_loss + weight * tf.reduce_mean(tf.abs(h1 - h2), axis=-1)
        #rc_loss = rc_loss + weight * tf.keras.backend.sum(tf.keras.backend.square(h1 - h2), axis=-1)

    return rc_loss

In [ ]:
def classification_loss(logit, label) :
    label= tf.cast(label, tf.float32)
    loss=loss_function(label, logit)
    
    return loss

In [34]:
def generator_loss(disc_generated_output, gen_output, target,Y_pred,target_label):
    percep_loss = perceptual_loss(target, gen_output)[0]
    gan_loss = loss_function(tf.ones_like(disc_generated_output), disc_generated_output)
    fake_c_loss = classification_loss(logit=Y_pred,label=target_label)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = (gan_loss) + 100*l1_loss + fake_c_loss + 10*percep_loss
    return total_gen_loss, gan_loss, l1_loss,fake_c_loss,target_label

In [35]:
def disc_layer(layer_input, filters, f_size=4, strides=2,normalization=False):
            
            init = RandomNormal(stddev=0.02)

            d_layer = ZeroPadding2D((1,1))(layer_input)
            d_layer = Conv2D(filters, kernel_size=f_size, strides=strides, padding='valid',kernel_initializer=init)(d_layer)
            if normalization:                
                d_layer = InstanceNormalization(axis=-1)(d_layer)
            d_layer = LeakyReLU(alpha=0.01)(d_layer)
            return d_layer

In [38]:
def discriminator():
        inp = Input(shape = [IMAGE_SIZE, IMAGE_SIZE, 3], name = "input_image")
        tar = Input(shape = [IMAGE_SIZE, IMAGE_SIZE, 3], name = "target_image")
        x = Concatenate()([inp, tar])
        
        model = disc_layer(x, 64, normalization=False)
        
        model = disc_layer(model, 128)
        model = disc_layer(model, 256)
        model = disc_layer(model, 512)
        model = disc_layer(model, 1024)
        model = disc_layer(model, 2048)        
        
        

        output_d_src = ZeroPadding2D((1,1))(model)

        output_d_src = Conv2D(1, kernel_size=3, strides=1, padding='valid')(output_d_src)
        

        model = Conv2D(filters =4,kernel_size = int(512/64), strides = 1,padding = 'valid')(model)      

        
        output_d_cls = Reshape((4,))(model)
        output_d_cls = tf.nn.softmax(output_d_cls)
        
        return Model(inputs = [inp, tar], outputs = [output_d_src,output_d_cls])


In [39]:
disc = discriminator()
disc.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 target_image (InputLayer)      [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 concatenate_8 (Concatenate)    (None, 512, 512, 6)  0           ['input_image[0][0]',            
                                                                  'target_image[0][0]']     

In [37]:
def discriminator_loss(disc_real_output, disc_generated_output,Y_true,target_label):
    real_loss = loss_function(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_function(tf.zeros_like(disc_generated_output), disc_generated_output)
    real_c_loss = classification_loss(logit=Y_true,label=target_label)
    total_disc_loss = real_loss + generated_loss + real_c_loss
    return total_disc_loss,real_c_loss

In [5]:
generator_optimizer = Adam(lr= 1e-4, beta_1=0.5,clipvalue=0.5)
discriminator_optimizer = Adam(lr = 1e-8, beta_1=0.5,clipvalue=0.5)

In [39]:
def save_images(model, test_input, target,label, epoch,exposure):
    prediction = model([test_input,label], training= True)
    gen.save('generator.h5')
    disc.save('discriminator.h5')
    if epoch%3==0:
        print(label)
        tf.keras.preprocessing.image.save_img(f"input_{epoch}_{exposure}.png",test_input[0])
        tf.keras.preprocessing.image.save_img(f"target_{epoch}_{exposure}.png",target[0])
        tf.keras.preprocessing.image.save_img(f"output_{epoch}_{exposure}.png",prediction[0])

In [40]:
epochs = 250

In [41]:
@tf.function
def train_step(input_image,target,target_label, epoch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = gen([input_image,target_label], training = True)
        disc_real_output,Y_true = disc([input_image, target], training = True)
        disc_generated_output,Y_pred = disc([input_image, gen_output], training = True)
        gen_total_loss, gen_gan_loss, gen_l1_loss,gcloss,target_label = generator_loss(disc_generated_output, gen_output, target,Y_pred,target_label)
        disc_loss, classification_loss = discriminator_loss(disc_real_output, disc_generated_output,Y_true,target_label)
        generator_gradients = gen_tape.gradient(gen_total_loss, gen.trainable_variables)
        discriminator_gradients = disc_tape.gradient(disc_loss, disc.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_gradients, gen.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, disc.trainable_variables))
        return gen_total_loss, disc_loss ,Y_true,Y_pred,gcloss,classification_loss,target_label

In [47]:
def fit(train_ds, epochs, test_ds):
    for epoch in range(30,epochs):
        start = time.time()
        for input_t, label in zip_test_dataset_0.take(1):
            input_np = np.array(input_t)
            target = input_np[1:2,:,:,:]
            input_ =input_np[0:1,:,:,:]
            target = tf.cast(target, tf.float32)
            input_ = tf.cast(input_, tf.float32)
            label =tf.reshape(label, [1, 4])
            save_images(gen, input_, target,label, epoch,2.5)
        for input_t, label in zip_test_dataset_1.take(1):
            input_np = np.array(input_t)
            target = input_np[1:2,:,:,:]
            input_ =input_np[0:1,:,:,:]
            target = tf.cast(target, tf.float32)
            input_ = tf.cast(input_, tf.float32)
            label =tf.reshape(label, [1, 4])
            save_images(gen, input_, target,label, epoch,3.0)
        for input_t, label in zip_test_dataset_2.take(1):
            input_np = np.array(input_t)
            target = input_np[1:2,:,:,:]
            input_ =input_np[0:1,:,:,:]
            target = tf.cast(target, tf.float32)
            input_ = tf.cast(input_, tf.float32)
            label =tf.reshape(label, [1, 4])
            save_images(gen, input_, target,label, epoch,3.5)
        for input_t, label in zip_test_dataset_3.take(1):
            input_np = np.array(input_t)
            target = input_np[1:2,:,:,:]
            input_ =input_np[0:1,:,:,:]
            target = tf.cast(target, tf.float32)
            input_ = tf.cast(input_, tf.float32)
            label =tf.reshape(label, [1, 4])
            save_images(gen, input_, target,label, epoch,4.0)      
        print(f"Epoch {epoch}")
        for n, (input_t, label) in train_ds.enumerate():
            input_np = np.array(input_t)
            target = input_np[1:2,:,:,:]
            input_ =input_np[0:1,:,:,:]
            target = tf.cast(target, tf.float32)
            input_ = tf.cast(input_, tf.float32)
            label =tf.reshape(label, [1, 4])
            gen_loss, disc_loss,Y_true,Y_pred,gcloss,classification_loss,target_label = train_step(input_, target,label, epoch)
        print("Generator loss {:.2f} Discriminator loss {:.2f}".format(gen_loss, disc_loss))
        print("Time take for epoch {} is {} sec\n".format(epoch+1, time.time() - start))

In [6]:
fit(zip_train_dataset, epochs, zip_test_dataset)